# Test the performance of the model

## Occurrence of symmetry equivalent structures

### Binary Quadratic Model

In [2]:
from quantum_computing_functions import *
from quantum_computing_postprocessing import *

from dwave.embedding.chain_strength import  uniform_torque_compensation

import numpy as np
import pandas as pd

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view

from pymatgen.ext.matproj import MPRester

'''import sys
sys.path.insert(1,'/Users/brunocamino/Desktop/Imperial/crystal-code-tools/crystal_functions/crystal_functions')
from file_readwrite import Crystal_gui
from convert import *'''

'''from crystal_functions.file_readwrite import *
from execute import runcry'''

'from crystal_functions.file_readwrite import *\nfrom execute import runcry'

In [35]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import copy
import numpy as np

lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

n_supercell = 3
scaling_matrix = np.identity(3)*n_supercell
scaling_matrix[2][2] = 1
graphene_supercell = copy.deepcopy(graphene)
graphene_supercell.make_supercell(scaling_matrix)
structure = graphene_supercell
graphene_supercell.num_sites

18

In [896]:
view(AseAtomsAdaptor().get_atoms(graphene_supercell))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/qc/bin/p...>

# Formation energy

In [18]:
with MPRester("p5vAQV3F1QuxFcxVT") as m:    
    graphite_pmg = m.get_structure_by_material_id("mp-48")

In [39]:
graphite_gui = cry_pmg2gui(graphite)

In [41]:
write_crystal_gui('./data/crystal/graphite.gui',graphite_gui)

In [28]:
SpacegroupAnalyzer(graphite_pmg).get_space_group_operations()

[Rot:
 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 1. -1.  0.]
  [ 1.  0.  0.]
  [ 0.  0.  1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[-1.  1.  0.]
  [-1.  0.  0.]
  [ 0.  0. -1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[ 0. -1.  0.]
  [ 1. -1.  0.]
  [ 0.  0.  1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  1.  0.]
  [-1.  1.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0.  1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0. -1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[-1.  1.  0.]
  [-1.  0.  0.]
  [ 0.  0.  1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 1. -1.  0.]
  [ 1.  0.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  1.  0.]
  [-1.  1.  0.]
  [ 0.  0.  1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[ 0. -1.  0.]
  [ 1. -1.  0.]
  [ 0.  0. -1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[ 1.  0.  0.]
  [ 1. -1.  0.]
  [ 0.  0. -1.]]
 tau
 [0.  0.  0.5],
 Rot:
 [[-1.  0.  0.]
  [-1.  1.  0.]
 

In [38]:
graphene_gui = cry_pmg2gui(graphene_supercell,dimensionality=2)
write_crystal_gui('./data/crystal/graphene.gui',graphene_gui)

In [48]:
graphene_1v = copy.deepcopy(graphene_supercell)
graphene_1v.remove_sites([0])
graphene_1v_gui = cry_pmg2gui(graphene_1v)
write_crystal_gui('./data/crystal/graphene_1v.gui',graphene_1v_gui)

In [55]:
E_graphite = Crystal_output().read_cry_output('data/crystal/graphite.out').get_final_energy()
E_graphene = Crystal_output().read_cry_output('data/crystal/graphene.out').get_final_energy()
E_graphene_1v = Crystal_output().read_cry_output('data/crystal/graphene_1v.out').get_final_energy()

In [57]:
E_graphite_atom = E_graphite/graphite.num_sites
E_graphene_atom = E_graphene/graphene.num_sites
E_graphite_1v_atom = E_graphite/graphene_1v.num_sites


In [58]:
E_graphene_atom

-9321.11994268472

In [59]:
E_graphite_1v_atom

-243.69344321171502

# Build discrete QUBO

## Vacancies

In [51]:
build_qubo_discrete_constraints(structure,[12,14],chem_potential=[-10,1],lambda_1 = 1, theta=10) 

array([[-20.,  20.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  -9.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0., -20.,  20.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  -9.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0., -20.,  20.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,  -9.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0., -20.,  20.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -9.]])

In [47]:
build_ip_matrix(structure, [12,14],parameters, alpha=100, max_neigh = 1)

IndexError: invalid index to scalar variable.

In [8]:
parameters = [-1,0,0]
q1 = build_ip_matrix(structure, [12,0], parameters, alpha=100, max_neigh = 1,test=True)+\
build_qubo_discrete_constraints(structure,[12,14],chem_potential=[1,0],lambda_1 = 1, theta=10) 
q3 = build_ip_matrix(structure, [12,0], parameters, alpha=100, max_neigh = 1,test=True)+\
build_qubo_discrete_constraints(structure,[12,14],concentration=[3,1],lambda_1 = 2, theta=10) 
q3

array([[-20.,  20.,   4.,   0.,   3.,   0.,   4.,   0.],
       [  0., -12.,   0.,   4.,   0.,   4.,   0.,   4.],
       [  0.,   0., -20.,  20.,   4.,   0.,   4.,   0.],
       [  0.,   0.,   0., -12.,   0.,   4.,   0.,   4.],
       [  0.,   0.,   0.,   0., -20.,  20.,   4.,   0.],
       [  0.,   0.,   0.,   0.,   0., -12.,   0.,   4.],
       [  0.,   0.,   0.,   0.,   0.,   0., -20.,  20.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., -12.]])

In [3]:
q2 = build_qubo_discrete_vacancies(structure,1, alpha = 1, lambda_1 = 2, theta=100)

In [9]:
q1 = build_ip_matrix(structure, [12,0], parameters, alpha=100, max_neigh = 1,test=True)+\
build_qubo_discrete_constraints(structure,[12,14],chem_potential=[0,-0.5],lambda_1 = 1, theta=10) 

#classical_solver(,q1,discrete=True)
q1

array([[-10. ,  20. ,   0. ,   0. ,  -1. ,   0. ,   0. ,   0. ],
       [  0. , -10.5,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. , -10. ,  20. ,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. , -10.5,   0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. , -10. ,  20. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. , -10.5,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. , -10. ,  20. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. , -10.5]])

In [193]:
from dimod import BinaryQuadraticModel
BinaryQuadraticModel.from_qubo(q1).num_variables

16

In [194]:
from dwave.system import EmbeddingComposite, DWaveSampler

sampler = EmbeddingComposite(DWaveSampler())

sampleset = sampler.sample_qubo(q1, num_reads=1000)

In [195]:
dataframe = convert_df(sampleset.to_pandas_dataframe()).head(50)
dataframe

,0,1,2,3,4,5,6,7,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,1,1,1,1,0.0,-92.0,153
1,1,1,1,1,1,1,1,0,0.0,-89.5,27
2,1,1,0,1,1,1,1,1,0.0,-89.5,52
3,1,1,1,1,0,1,1,1,0.0,-89.5,24
4,0,1,1,1,1,1,1,1,0.0,-89.5,25
5,1,0,1,1,1,1,1,1,0.0,-89.5,44
6,1,1,1,0,1,1,1,1,0.0,-89.5,7
7,1,1,1,1,1,1,0,1,0.0,-89.5,24
8,1,1,1,1,1,0,1,1,0.0,-89.5,23
9,0,1,1,1,0,1,1,1,0.0,-88.0,9


In [164]:
np.sum(dataframe.iloc[:,0:18].to_numpy(),axis=1)

array([-68., -72., -67., -65., -72., -70., -67., -67., -69., -70., -69.,
       -71., -68., -73., -70., -69., -71., -64., -69., -69., -66., -62.,
       -64., -68., -58., -69., -68., -68., -68., -72., -64., -67., -70.,
       -74., -71., -69., -67., -69., -71., -65., -69., -71., -63., -73.,
       -72., -62., -75., -62., -63., -71.])

In [111]:
configurations = dataframe.iloc[:,0:50].to_numpy()
multiplicity = dataframe['num_occurrences'].to_numpy()
energies = dataframe['energy'].to_numpy()

In [112]:
all_structures = []
for config in configurations:
    structure_2 = copy.deepcopy(graphene_supercell)
    for j in np.where(config == 0)[0]:
        structure_2.replace(j,1)
    all_structures.append(structure_2)
    

In [114]:
from pymatgen.analysis.structure_matcher import StructureMatcher 

unique_structures = StructureMatcher().group_structures(all_structures)
unique_structures_label = []

for struc in all_structures:
    for i in range(len(unique_structures)):
        #print(unique_structures[i][0].composition.reduced_formula,structure.composition.reduced_formula)
        if StructureMatcher().fit(struc,unique_structures[i][0]) == True:
            unique_structures_label.append(i)
            break

In [115]:
len(unique_structures)

50

In [117]:
for i in range(len(unique_structures[0:10])):
    print(unique_structures[i][0].composition.reduced_formula)
    view(AseAtomsAdaptor().get_atoms(unique_structures[i][0]))

HC24
H3C47
H2C23
H2C23
H2C23
HC9
HC9
HC9
HC9
HC9


In [58]:
df = sampleset.to_pandas_dataframe()
def convert_df(df,remove_unfeasible=True):
    
    num_sites = sum([type(x) == int for x in df.columns])
    sites = df.iloc[:,0:num_sites].to_numpy()
    unfeasible = np.where(np.prod((sites[:,::2]+sites[:,1::2]),axis=1) != 1)[0]
    df.drop(unfeasible, inplace=True)
    
    new_labels = df.iloc[:,0:num_sites].to_numpy()[:,::2]
    df.drop(np.arange(num_sites),axis=1,inplace=True)

    
    for i in range(new_labels.shape[1]):
        df.insert(i, i, new_labels[:,i])
    
    return df
convert_df(df)

,0,1,2,3,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,0.0,-44.0,267
1,1,0,1,1,0.0,-42.5,115
2,1,1,1,0,0.0,-42.5,69
3,1,1,0,1,0.0,-42.5,61
4,0,1,1,1,0.0,-42.5,79
5,1,0,0,1,0.0,-42.0,59
6,0,0,0,0,0.0,-42.0,44
7,0,1,0,1,0.0,-42.0,41
8,0,1,1,0,0.0,-42.0,47
9,1,0,1,0,0.0,-42.0,64


In [104]:
import itertools
x = np.array(list(itertools.product([0, 1], repeat=4)))
xx = np.array([[0,1]*structure.num_sites]*x.shape[0])
sorting = np.argsort(np.sum(x,axis=1))
        
x = x[sorting]
np.append(np.array(x)[:],np.array(x)[:],axis=1)

X = np.zeros((x.shape[0],2*x.shape[1]),dtype=int)
for i in range(x.shape[0]):
    X[i] = np.vstack([x[i,:], x[i,:]]).flatten(order='F')

x_discrete = np.array(np.logical_xor(X,xx),dtype=int)

In [108]:
classical_solver()

-416.0

In [110]:
x_discrete[-2]

array([1, 0, 1, 0, 1, 0, 0, 1])

In [172]:
def build_ip_matrix(structure, species, parameters, alpha=100, max_neigh = 1,test=False):
    
    num_sites = structure.num_sites
    num_species = len(species)
    num_elements = num_sites*num_species
    
    distance_matrix = np.round(structure.distance_matrix,5)
    shells = np.unique(np.round(distance_matrix,5))
    
    if test == True:
        adjacency_matrix = build_adjacency_matrix(structure)
        param = [0,1,0,0]
    
    # Generate an all False matrix
    distance_matrix_filter = (distance_matrix == -1)

    
    # Only add the atoms within the shells up to max_neigh 
    for neigh in range(1,max_neigh+1):
        distance_matrix_filter +=  distance_matrix == shells[neigh]  

    # Buckingham
    #loop 
    
    ip_matrix = np.zeros((num_elements,num_elements))
    parameters = np.array(parameters)
    for i in range(num_sites):
        for j in range(i,num_sites):
            if distance_matrix_filter[i,j] == True:
                index = -1
                for k in range(num_species):
                    for l in range(k,num_species):
                        index += 1
                        param = parameters[index]
                        #print(i*num_species+k,j*num_species+l)
                        if test == True:
                            ip_matrix[i*num_species+k,j*num_species+l] = \
                            adjacency_matrix[i,j]*param
                        else:
                            ip_matrix[i*num_species+k,j*num_species+l] = \
                            param[0] * np.exp((-distance_matrix[i,j])/(param[1]))- \
                            ((param[2])/((distance_matrix[i,j])**6))
                        
                
    return np.round(ip_matrix,2)

In [72]:
build_qubo_matrix(build_bqm_discrete(structure,[12,14],[0.5,0.5], parameters, alpha=1, lambda_1=2, theta=10, max_neigh = 1))

[[-10.    20.     4.     0.     4.24   0.98   4.24   0.98]
 [  0.   -10.     0.     4.     0.     4.24   0.     4.24]
 [  0.     0.   -10.    20.     4.24   0.98   4.24   0.98]
 [  0.     0.     0.   -10.     0.     4.24   0.     4.24]
 [  0.     0.     0.     0.   -10.    20.     4.     0.  ]
 [  0.     0.     0.     0.     0.   -10.     0.     4.  ]
 [  0.     0.     0.     0.     0.     0.   -10.    20.  ]
 [  0.     0.     0.     0.     0.     0.     0.   -10.  ]]


array([[-10,  20,   4,   0,   4,   0,   4,   0],
       [  0, -10,   0,   4,   0,   4,   0,   4],
       [  0,   0, -10,  20,   4,   0,   4,   0],
       [  0,   0,   0, -10,   0,   4,   0,   4],
       [  0,   0,   0,   0, -10,  20,   4,   0],
       [  0,   0,   0,   0,   0, -10,   0,   4],
       [  0,   0,   0,   0,   0,   0, -10,  20],
       [  0,   0,   0,   0,   0,   0,   0, -10]])

In [71]:
parameters = [[2457.243, 0.2610,  0.00],
             [2457.243, 0.2610,  0.00],
             [2457.243, 0.2610,  0.00]]
             
parameters = [[1.,1.,0.],
            [2,2,0.],
             [1,1,0.]]
             

#build_bqm_discrete(structure,species,concentrations, parameters, weight=100, max_neigh = 1)
#build_bqm_discrete(structure,[12,14],[0.5,0.5], parameters, weight=100, max_neigh = 1)

In [891]:
df = sampleset.to_pandas_dataframe()
def convert_df(df,remove_unfeasible=True):
    
    num_sites = sum([type(x) == int for x in df.columns])
    sites = df.iloc[:,0:num_sites].to_numpy()
    unfeasible = np.where(np.prod((sites[:,::2]+sites[:,1::2]),axis=1) != 1)[0]
    df.drop(unfeasible, inplace=True)
    
    new_labels = df.iloc[:,0:num_sites].to_numpy()[:,::2]
    df.drop(np.arange(num_sites),axis=1,inplace=True)
    
    new_col = {}
    for i in range(int(num_sites/2)):
        new_col[i] = new_labels[:,i].tolist()
    return df
convert_df(df)

,chain_break_fraction,energy,num_occurrences
0,0.000000,-2316.0,1
1,0.000000,-2314.0,1
2,0.000000,-2314.0,2
3,0.000000,-2314.0,1
4,0.000000,-2314.0,1
...,...,...,...
945,0.027778,-2203.0,1
959,0.027778,-2162.0,1
981,0.055556,-2271.0,1
985,0.055556,-2207.0,1


In [717]:
ar = df.iloc[:,0:36].to_numpy()
np.sum(df['num_occurrences'].to_numpy()[np.where(np.sum(ar[:,::2],axis=1) == 17)[0]])
ar.shape

(490, 36)

In [749]:
keep = np.where(np.sum((ar[:,::2]+ar[:,1::2])-np.ones((ar.shape[0],int(ar.shape[1]/2))),axis=1) == 0. )
not_keep = np.where(np.sum((ar[:,::2]+ar[:,1::2])-np.ones((ar.shape[0],int(ar.shape[1]/2))),axis=1) != 0. )[0]
#aa = np.sum((ar[:,::2]+ar[:,1::2])-np.ones((ar.shape[0],int(ar.shape[1]/2))),axis=1) 
print(np.where(np.sum(ar[keep][:,::2],axis=1) == 17))
not_keep2 = np.where(np.sum(ar[keep][:,::2],axis=1) != 17)[0]
np.sum(df['num_occurrences'].to_numpy()[keep][np.where(np.sum(ar[keep][:,::2],axis=1) == 17)])
df.drop(not_keep)
df.drop(not_keep2)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        78,  79,  80,  82,  83, 111, 112, 117, 118, 119, 120, 121, 124,
       125, 126, 128, 129, 130, 131, 134, 136, 139, 140, 142, 157, 158,
       160, 161, 162, 166, 170, 171, 172, 174, 178, 179, 180, 181, 184,
       187, 190, 191, 192, 195, 208, 209, 210, 218, 226, 228, 229, 230]),)


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,chain_break_fraction,energy,num_occurrences
0,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0.000000,-760.0,27
1,0,1,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0.000000,-760.0,8
2,1,0,1,0,1,0,1,0,0,1,...,0,1,0,1,0,1,0,0.000000,-760.0,34
3,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0.000000,-760.0,42
4,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0.000000,-760.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,1,0,1,0,1,0,0,1,1,0,...,0,0,0,1,0,1,0,0.083333,-732.0,1
486,1,0,1,0,1,0,1,0,1,1,...,0,1,0,1,0,1,0,0.083333,-736.0,1
487,1,0,0,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0.027778,-708.0,1
488,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0.055556,-748.0,1
